## This is a Jupyter notebook to optimise a unit cell structure and create a supercell using the MACE-MP-0 interatomic potential and the Atomic Simulation Environment (ASE).

Sections that require user input for a specific system are shown as ################## User input ################# blocks.

### References:

#### ASE:

https://wiki.fysik.dtu.dk/ase/ase/io/io.html
https://wiki.fysik.dtu.dk/ase/ase/neb.html


#### MACE: 

Code: https://github.com/ACEsuit/mace-mp?tab=readme-ov-file

Paper: Batatia, I., Benner, P., Chiang, Y., Elena, A.M., Kovács, D.P., Riebesell, J., Advincula, X.R., Asta, M., Baldwin, W.J., Bernstein, N. and Bhowmik, A., 2023. A foundation model for atomistic materials chemistry. arXiv preprint arXiv:2401.00096.

## Step 0: Import some modules from ASE and MACE

In [1]:
import ase
from ase.optimize import LBFGS
from ase.io import read, write
from ase.filters import FrechetCellFilter
from ase.build import make_supercell
from ase.io.trajectory import Trajectory
from ase import Atoms
from ase.constraints import FixAtoms
from ase.visualize import view
from mace.calculators import mace_mp

## Step 1: Import structure
Next, we will read in an initial structure file in the same directory called 'FePO4.cif'. This file, which is in the cif format, can be from experimental databases, such as the ICSD, or computational databases such as the Materials Project (https://next-gen.materialsproject.org/) or OQMD (https://oqmd.org/). Other file formats can also be read by ase (https://wiki.fysik.dtu.dk/ase/ase/io/io.html). To read in a VASP 'POSCAR' file change use format='vasp'.

In [2]:
################## User input #################
initial_structure=read('FePO4.cif',format='cif')
################## User input #################

In [3]:
view(initial_structure, viewer='x3d')

Once the structure has been read in, the mace_mp calculator is assigned to it.

In [4]:
macemp = mace_mp(default_dtype="float32")
initial_structure.calc=macemp

Using Materials Project MACE for MACECalculator with /Users/ieuanseymour/.cache/mace/5yyxdm76
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


## Step 2: Optimise the unit cell

Now we want to optimise the structure with MACE-MP-0 model in ase. To allow the unit cell parameters to relax as well as the atomic positions, we need to apply a filter. We then use the LBFGS optimiser to optimise the atom positions and the lattice parameters until the force on any atom falls below 0.01 eV/A. The changes in the atomic positions and the lattice parameters is captured in the 'unit_cell_opt.traj' trajectory file, which can be read by the ase gui package. 

In [5]:
initial_structure.set_constraint(FixAtoms(mask=[True for atom in initial_structure]))

ecf=FrechetCellFilter(initial_structure)
bfgs=LBFGS(ecf)
traj = Trajectory('unit_cell_opt.traj', 'w', initial_structure)
bfgs.attach(traj)
bfgs.run(fmax=0.01)

       Step     Time          Energy          fmax
LBFGS:    0 00:03:44     -169.744553        0.108365
LBFGS:    1 00:03:46     -169.744797        0.106031
LBFGS:    2 00:03:47     -169.749878        0.012782
LBFGS:    3 00:03:49     -169.749878        0.013168
LBFGS:    4 00:03:50     -169.749893        0.013824
LBFGS:    5 00:03:51     -169.749939        0.012916
LBFGS:    6 00:03:52     -169.749969        0.008835


True

## Step 3: Create a supercell

Create supercell structure of the optimised unit cell and write the structure to a cif file 'supercell.cif'. For this example a (1x2x2) supercell is used, in which the b and c lattice parameters of the FePO4 unit cell are doubled.

In [6]:
################## User input #################
super_matrix=[[1,0,0],[0,2,0],[0,0,2]]
################## User input #################

In [7]:
fully_occupied_super=make_supercell(initial_structure,super_matrix)
view(fully_occupied_super, viewer='x3d')

In [8]:
write('supercell.cif',fully_occupied_super,format='cif')